This script adds (1) Flash Repeats and (2) Block numbers (3) Block_mean_response to a panda dataframe with information from flash_responses and all_trials. 

Please note the step of including dateTime to identify novel sessions, necessary per our analysis design. 

August 2018
Created by M. Sol Bernardez Sarria in collaboration with project teammates and SWDB_2018 TAs


In [ ]:
import matplotlib
%matplotlib notebook

import os
import numpy as np
import pandas as pd

import sys

import matplotlib as mpl
import matplotlib.pyplot as plt

# this makes print statements complaint with python 3 formatting
from __future__ import print_function

from datetime import datetime

#Behavior
from visual_behavior.ophys.dataset.visual_behavior_ophys_dataset import VisualBehaviorOphysDataset
from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis 

# Local of the event drive path
event_drive_path_AWS = '/data/dynamic-brain-workshop/visual_behavior_events' #AWS Location
drive_path =  '/data/dynamic-brain-workshop/visual_behavior'

#from functions developed by/with our group
import oBehave.helperfunctions as obhf
import oBehave.behavior_anaylsis as obba
import oBehave.perCell_functions as obpc

In [ ]:
#load the manifest file for all experiments
#Below we subselected for a particular area (VISp) and genotype (Slc17a7-IRES2-Cre)
manifest = obhf.load_manifest()

#add dateTime > add Novel session
#this step was crucial. Our analysis framework required considering the first exposure to
#Image Sets B, C & D only (which was not always the case and could not be assumed)
obba.includeDateTime(manifest,dateStringKey = 'experiment_date',datetimeKey = 'experiment_datetime',dateConversionStr = '%m/%d/%Y %H:%S')
obba.includeNovelSession(manifest)


#create mask
mask_AreaGenoFirst = (manifest.targeted_structure=='VISp')&(manifest.cre_line=='Slc17a7-IRES2-Cre')&(manifest.first_session==True)
#load relevant data
data_inv = manifest[mask_AreaGenoFirst] 

#display df for double-checking
data_inv.head()


In [ ]:
#identify unique experimentIDs within the data
allexpIDs = data_inv.experiment_id.unique()
len(allexpIDs)

Below is a for loop to include:
(1) Flash/Repeats  
(2) Block number to the data, and
(3) Average mean_response overt the first 3 Flahes/Repeats within each Block

New df includes only our columns of interest (modifiable within this for loop), and is saved as a csv, separately per expID

The Block mean response function can be modified to include more/less repeats. It's important to note that the output of this function will be repeated for every Flash within a Black for reference (if you want to compare the response to Flash 7 and the mean of the first 3 Flashes, you could do so within the same row)

Functions are in perCell_functions.py

Note: this will take a while, prints every time a loop ends

In [ ]:
#for loop to include Flash Repeats and Block number to the data, 
#subselect a data frame with the wanted information
#Add mean_response to first 3 Flashes within a Block
#save to a csv file to read when necessary
counter = 1
for expID in allexpIDs:
    expID = str(expID)
    dataset= VisualBehaviorOphysDataset(experiment_id=expID, cache_dir=obhf.drive_path)
    analysis = ResponseAnalysis(dataset)
    
    data_everything = obpc.include_add_repeats(analysis.flash_response_df, dataset.all_trials)
    obpc.validate_resets(data_everything)
    obpc.validate_block_transitions(data_everything)

    data = data_everything.loc[:,['cell','flash_number', 
                       'start_time', 'image_name','mean_response', 
                        'p_value', 'block', 'repeats']].copy()
    
    temp = obpc.get_block_mean_response(data)
    tfilename = ('data_wBlock_mean:', str(expID), '.csv')
    filename = ''.join(tfilename)
    temp.to_csv(filename)
    print('loop done', counter)
    counter = counter+1
    del(tfilename, filename)

In [ ]:
filename = ('data_manifest.pkl')
data_inv.to_pickle(filename)